In [1]:
import os
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.onnx
import onnx

from google.protobuf import json_format

# First we export a PyTorch model
filename = "model.proto"
fullpath = os.path.join("/tmp", filename)

dummy_input = Variable(torch.randn(2, 5))
model = torch.nn.Sequential(
          torch.nn.Linear(5, 4),
#           torch.nn.Dropout(p=0.9),
          torch.nn.ReLU(),
          torch.nn.Linear(4, 3),
#           torch.nn.Dropout(p=0.9),
          torch.nn.ReLU(),
          torch.nn.Linear(3, 2),
          torch.nn.Softmax(dim=1),
        )
torch.onnx.export(model, dummy_input, fullpath)
print("model saved at: %s" % fullpath)


model saved at: /tmp/model.proto


In [2]:
import syft
import syft.nn
import syft.onnx
from syft import FloatTensor

/Users/morgangiraud/.virtualenvs/openmined/lib/python3.6/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [3]:
# Load the PyTorch model
syft_dummy_input = FloatTensor([[1,2,3,4,5],[5,4,3,2,1]], autograd=True)
syft_model = syft.onnx.load(fullpath)

syft_model.summary()
# Execute models
pytorch_output = model(dummy_input)
syft_output = syft_model(syft_dummy_input)
print(pytorch_output)
print(syft_output)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
linear_2 (Linear)            (None, 4)                 24
_________________________________________________________________
relu_3 (ReLU)                (dynamic)                 0
_________________________________________________________________
linear_4 (Linear)            (None, 3)                 15
_________________________________________________________________
relu_5 (ReLU)                (dynamic)                 0
_________________________________________________________________
linear_6 (Linear)            (None, 2)                 8
_________________________________________________________________
softmax_7 (Softmax)          (dynamic)                 0
Total params: 47
Trainable params: 47
Non-trainable params: 0
_________________________________________________________________

Variable containing:
 0.3928  0.6072
 0.3939  0.6061
[torch.Float

In [4]:
# Testing consistency between serialization and deserialization in OpenMined

syft.onnx.export(syft_model, syft_dummy_input, fullpath)
new_syft_model = syft.onnx.load(fullpath)

syft_output = syft_model(syft_dummy_input)
new_syft_output = new_syft_model(syft_dummy_input)

print(syft_output)
print(new_syft_output)

   0.4015207  0.5984793 
   0.412447   0.587553   
   0.4015207  0.5984793 
   0.412447   0.587553   
